In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from scipy.signal import find_peaks, butter, filtfilt

# Constants
SAMPLING_RATE = 500  # Hz
ECG_FOLDER = "../../../Datasets/12-lead electrocardiogram database/ECGData"
DIAGNOSTICS_FILE = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Rhythm Mapping
RHYTHM_MAPPING = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

# Hamilton-Tompkins QRS Detection
def preprocess_ecg(ecg_signal, sampling_rate=SAMPLING_RATE):
    """Preprocess ECG data: Bandpass filter and detect R-peaks."""
    if len(ecg_signal) == 0:
        return [], ecg_signal

    # Bandpass filter (0.5 - 50 Hz)
    lowcut = 0.5
    highcut = 50.0
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)

    # Derivative
    derivative = np.diff(filtered_ecg)

    # Squaring
    squared = derivative ** 2

    # Moving window integration
    window_size = int(0.150 * sampling_rate)  # 150ms
    integrated = np.convolve(squared, np.ones(window_size) / window_size, mode='same')

    # Find R-peaks
    r_peaks, _ = find_peaks(integrated, distance=sampling_rate * 0.6, height=np.mean(integrated))
    return r_peaks, filtered_ecg

def extract_features(ecg_data, r_peaks):
    """Extract features from ECG data and R-peaks."""
    # Initialize feature dictionary
    features = {
        "RRMean": 0,
        "RRStd": 0,
        "VentricularRate": 0,
        "QRSCount": 0,
        "QRSDuration": 0,
        "QTInterval": 0,
        "QTCorrected": 0,
        "RAxis": 0,
        "TAxis": 0,
        "QOnset": 0,
        "QOffset": 0,
        "TOffset": 0
    }

    if len(r_peaks) > 1:
        rr_intervals = np.diff(r_peaks) / SAMPLING_RATE * 1000  # in ms
        features["RRMean"] = np.mean(rr_intervals)
        features["RRStd"] = np.std(rr_intervals)
        features["VentricularRate"] = 60000 / features["RRMean"] if features["RRMean"] > 0 else 0
    features["QRSCount"] = len(r_peaks)

    # Handle onset/offset computation safely
    q_onset = np.clip(r_peaks - int(0.05 * SAMPLING_RATE), 0, len(ecg_data) - 1)
    q_offset = np.clip(r_peaks + int(0.05 * SAMPLING_RATE), 0, len(ecg_data) - 1)
    t_offset = np.clip(r_peaks + int(0.35 * SAMPLING_RATE), 0, len(ecg_data) - 1)

    if len(r_peaks) > 0:
        features["QRSDuration"] = np.mean(q_offset - q_onset) / SAMPLING_RATE * 1000  # in ms
        features["QTInterval"] = np.mean(t_offset - q_onset) / SAMPLING_RATE * 1000  # in ms
        features["QTCorrected"] = features["QTInterval"] / (features["RRMean"] ** 0.5) if features["RRMean"] > 0 else 0
        features["QOnset"] = np.mean(q_onset)
        features["QOffset"] = np.mean(q_offset)
        features["TOffset"] = np.mean(t_offset)

    # Simplified R/T axis (placeholder values)
    features["RAxis"] = np.mean(ecg_data[q_onset]) if len(q_onset) > 0 else 0
    features["TAxis"] = np.mean(ecg_data[t_offset]) if len(t_offset) > 0 else 0

    return features

# Load and preprocess data
diagnostics_df = pd.read_excel(DIAGNOSTICS_FILE)
diagnostics_df['MappedRhythm'] = diagnostics_df['Rhythm'].map(RHYTHM_MAPPING)

features = []
labels = []

for idx, row in diagnostics_df.iterrows():
    try:
        ecg_file = os.path.join(ECG_FOLDER, row['FileName'] + '.csv')  # Add .csv extension
        if os.path.exists(ecg_file):
            ecg_data = pd.read_csv(ecg_file, skiprows=1, usecols=[1]).to_numpy().flatten()
            if len(ecg_data) > 0:
                r_peaks, filtered_ecg = preprocess_ecg(ecg_data)
                feature = extract_features(filtered_ecg, r_peaks)
                features.append(feature)
                labels.append(row['MappedRhythm'])
    except Exception as e:
        print(f"Error processing file {row['FileName']}: {e}")

# Convert to DataFrame
features_df = pd.DataFrame(features)
features_df['Label'] = labels

# Encode labels
label_encoder = LabelEncoder()
features_df['EncodedLabel'] = label_encoder.fit_transform(features_df['Label'])

# Split data
X = features_df.drop(['Label', 'EncodedLabel'], axis=1).values
y = features_df['EncodedLabel'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6950, stratify=y)

# Normalize features for MLP
X_train_norm = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test_norm = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=6950)
dt_classifier.fit(X_train, y_train)
dt_preds = dt_classifier.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_preds, target_names=label_encoder.classes_))

# MLP Classifier
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train_norm, y_train, epochs=50, batch_size=32, validation_split=0.2)

mlp_preds = np.argmax(mlp_model.predict(X_test_norm), axis=1)
print("MLP Classification Report:")
print(classification_report(y_test, mlp_preds, target_names=label_encoder.classes_))

Decision Tree Classification Report:
              precision    recall  f1-score   support

        AFIB       0.58      0.59      0.58       445
        GSVT       0.58      0.60      0.59       462
          SB       0.95      0.96      0.95       778
          SR       0.85      0.80      0.82       445

    accuracy                           0.77      2130
   macro avg       0.74      0.74      0.74      2130
weighted avg       0.77      0.77      0.77      2130

Epoch 1/50


/tmp/ipykernel_638298/3615098329.py:139: RuntimeWarning: invalid value encountered in divide
  X_train_norm = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
/tmp/ipykernel_638298/3615098329.py:140: RuntimeWarning: invalid value encountered in divide
  X_test_norm = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


213/213 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.2084 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 2/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.2053 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 3/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2067 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 4/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.2172 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 5/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2069 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 6/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2099 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 7/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2102 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 8/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2105 - loss: nan - val_accuracy: 0.1966 - val_loss: nan
Epoch 9/50
213/213 ━━━━━━━━━━━

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control